In [ ]:
import pandas as pd
import numpy as np
import os
my_path = os.path.abspath('') 


In [ ]:
def group_consecutive_numbers(x):
    if not x:
        return []

    grouped_list = []
    group_number = 1
    current_group_label = f'g{group_number}'
    grouped_list.append(current_group_label)

    for i in range(1, len(x)):
        # Check if the current number is consecutive to the previous one
        if x[i] == x[i-1] + 1:
            grouped_list.append(current_group_label)
        else:
            # If not consecutive, start a new group
            group_number += 1
            current_group_label = f'g{group_number}'
            grouped_list.append(current_group_label)
    return grouped_list

def xai(list_pattern):
    if type(list_pattern) != str:
        return 'normal'
    else:
        list_pattern = np.sort(eval(list_pattern))
        lp = []
        ll = []
        la = []
        for el in list_pattern:
            els = el.split('_')
            if len(els) == 3:
                lp.append(els[0])
                la.append(els[2])
                if els[0] == 'replace':
                    ll.append(0)
                else:
                    ll.append(int(els[1]))
            elif len(els) ==4: #move
                lp.append(els[0])
                ll.append(int(els[2]))
                la.append(els[3])

        # lg = group_consecutive_numbers(ll)
        df_temp = pd.DataFrame({'p':lp, 'l':ll, 'a': la})
        df_temp = df_temp.sort_values(by=['l']).reset_index(drop=True)
        lg = group_consecutive_numbers(df_temp['l'].tolist())
        df_temp['g'] = lg
        pred = []

        if 'rework' in lp:
            df_temp2= df_temp.loc[df_temp.p == 'rework']
            pred = pred+ df_temp2[['p','a']].drop_duplicates()['p'].tolist()

        if 'insert' in lp:
            df_temp2= df_temp.loc[df_temp.p == 'insert']
            pred = pred+ df_temp2[['p','g']].drop_duplicates()['p'].tolist()

        if 'move' in lp:
            df_temp2= df_temp.loc[df_temp.p == 'move'].reset_index(drop =True)
            df_temp2.p = 'moved'
            df_temp2 = df_temp2[df_temp2.index % 2 == 0]
            pred = pred+ df_temp2[['p','a','g']].drop_duplicates()['p'].tolist()

        if 'replace' in lp:
            df_temp2= df_temp.loc[df_temp.p == 'replace'].reset_index(drop =True)
            if len(df_temp2)%2 == 1:

                filtered_list = [item for item in list_pattern if 'replace' in item]
                filtered_list_a = [item for item in filtered_list if 'inject' in item]
                if len(filtered_list_a) > (len(filtered_list)-len(filtered_list_a)):
                    pred= pred + ['skip']
                else:
                    pred = pred + ['insert']
            df_temp2 = df_temp2[df_temp2.index % 2 == 1]
            pred = pred+ df_temp2[['p','a','g']].drop_duplicates()['p'].tolist()

        if 'skip' in lp:
            df_temp2= df_temp.loc[df_temp.p == 'skip']
            pred = pred+ df_temp2['p'].tolist()
            df_temp3= df_temp.loc[df_temp.p == 'insert']

            for loc1 in df_temp2['l'].tolist():
                if sum(np.abs(df_temp3['l'] -loc1) ==1) > 0:
                    pred = pred + ['replace']
                    if sum(np.arry(pred)=='insert')>0:
                        pred.remove('insert')
                    pred.remove('skip')

        pred= np.sort(pred)
        pred = '+'.join(pred)
        return pred

In [ ]:
output_path = os.path.join(my_path, "output")
list_data =os.listdir(output_path)
list_data

# Predict patterns and save it as a new data column

In [ ]:
for dat in list_data:
    anomalous_data_path = os.path.join(output_path, dat)
    data = pd.read_csv(anomalous_data_path)

    df1 = data[['case:concept:name', 'predict_patterns']].drop_duplicates().reset_index(drop=True)
    df1['refined_patterns'] = df1['predict_patterns'].apply(lambda x: xai(x))
    data = pd.merge(data, df1[['case:concept:name', 'refined_patterns']], on = 'case:concept:name')
    data.to_csv(anomalous_data_path, index= False)